# Segmenting and Clustering Neighborhoods in Toronto
#### The task of this assignment is to explore, segment, and cluster the neighborhoods in the city of Toronto. The task can be classified into three  parts. The first part is to scrape the data from web and after that it is required to clean the data. Clustering the different neigberhoods of Toronto is the third task of this assignment

### Importing Librairies 

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium 
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## Part 1: Scraping the data from web

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(source.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")
rows=table.findAll('tr')
rows=rows[1:len(rows)]
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")
    
df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [3]:
df["Postal Code"] = df["Postal Code"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df.head()


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Ignoring cells with "Not Assigned" boroughs and putting a "Not Assigned" neigberhood as respective borough

In [29]:
df = df.drop(df[(df.Borough == "Not assigned")].index)

df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)

df.Neighborhood.fillna(df.Borough, inplace=True)

df=df.drop_duplicates()
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
#Grouping by Boroughs
df = df.groupby(["Postal Code", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [31]:
df.shape

(103, 3)

# Part 2: Data cleaning

In [32]:

toronto_geo = 'https://cocl.us/Geospatial_data'
df_geo = pd.read_csv(toronto_geo)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging the two tables

In [33]:
df_geo.drop("Postal Code", axis=1, inplace=True)
df_Toronto = df.join(df_geo)
df_Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Map of Toronto

In [24]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [34]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(
        df_Toronto['Latitude'], 
        df_Toronto['Longitude'], 
        df_Toronto['Borough'], 
        df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

### Creating a new data frame for Scarborough

In [36]:
scarborough_data = df_Toronto[df_Toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Map of Scarborough

In [25]:
#Geographical cordinates of Scarborough
address = 'Scarborough,Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [28]:
map_scarb = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb



#### Four square credential

In [37]:
CLIENT_ID = 'UDGM4KUSCGFGHH5HCH5OOSSHLOJXW4BM0IIKV000EBFUCT5I' # your Foursquare ID
CLIENT_SECRET = 'XTV4ARDERPWJRP01XCCUTALFT0WT4MYKSZZSIBCJ3GVAJYI0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UDGM4KUSCGFGHH5HCH5OOSSHLOJXW4BM0IIKV000EBFUCT5I
CLIENT_SECRET:XTV4ARDERPWJRP01XCCUTALFT0WT4MYKSZZSIBCJ3GVAJYI0


#### exploring the first neighborhood in our dataframe.
#####  neighborhood's name  and its Long and Lat.

In [39]:
scarborough_data.loc[0, 'Neighborhood']

'Malvern, Rouge'

In [41]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


### Top 100 veneus in radius of 500m

In [43]:
LIMIT = 100 

radius = 500 


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=UDGM4KUSCGFGHH5HCH5OOSSHLOJXW4BM0IIKV000EBFUCT5I&client_secret=XTV4ARDERPWJRP01XCCUTALFT0WT4MYKSZZSIBCJ3GVAJYI0&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [44]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5efe106fe771111ff8474a44'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [45]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [46]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [47]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


## Analyzing neigberhoods in Scarborough

In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [52]:
print(scarborough_venues.shape)
scarborough_venues.head()

(88, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [53]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",13,13,13,13,13,13
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile, Clairlea, Oakridge",7,7,7,7,7,7
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


In [54]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 54 uniques categories.


In [55]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [57]:
scarborough_onehot.shape

(88, 55)

In [58]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.2,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.0,0.000000,0.000000,0.000,0.000000,0.00,0.2,0.000000,0.2,0.000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.20,0.000000,0.000000,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.25,0.000,0.000000,0.0,0.000000,0.25,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000,0.0,0.000000,0.000000,0.000,0.000000,0.00,0.0,0.000000,0.0,0.000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000
2,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.125,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.125000,0.000000,0.125000,0.00,0.000000,0.125,0.0,0.000000,0.000000,0.000,0.000000,0.00,0.0,0.000000,0.0,0.000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.125000,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000,0.000000,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.076923,0.0,0.000000,0.00,0.0,0.000000,0.000,0.076923,0.076923,0.000000,0.076923,0.00,0.000000,0.000,0.0,0.000000,0.000000,0.000,0.076923,0.00,0.0,0.000000,0.0,0.000,0.000000,0.000,0.000000,0.0,0.076923,0.000000,0.000000,0.153846,0.153846,0.0,0.076923,0.000,0.000000,0.00,0.000000,0.000000,0.076923,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.0,0.000000,0.000000,0.000,0.000000,0.00,0.0,0.000000,0.0,0.000,0.000000,0.000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.142857,0.0,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.000000,0.142857,0.000000,0.00,0.000000,0.000,0.0,0.000000,0.285714,0.000,0.000000,0.00,0.0,0.142857,0.0,0.000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000,0.000000,0.285714,0.000000,0.0,0.000000,0.000000,0.285714,0.0,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.0,0.142857,0.000000,0.000,0.000000,0.00,0.0,0.000000,0.0,0.000,0.142857,0.000,0.000000,0.0,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.0,0.000,0.000000,0.000000,0.125000,0.0,0.125000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000

### Top 5 most common veneus in each neigberhood

In [59]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1               Skating Rink   0.2
2             Breakfast Spot   0.2
3                     Lounge   0.2
4             Clothing Store   0.2


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4    American Restaurant  0.00


----Cedarbrae----
                  venue  freq
0      Hakka Restaurant  0.12
1  Caribbean Restaurant  0.12
2       Thai Restaurant  0.12
3                Bakery  0.12
4                  Bank  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0              Pharmacy  0.15
1           Pizza Place  0.15
2   Rental Car Location  0.08
3  Fast Food Restaurant  0.08
4          Noodle House  0.08


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1 

### Data frame for top 10 venues in each neigberhood

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,Grocery Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Pharmacy,Fast Food Restaurant,Noodle House,Italian Restaurant,Rental Car Location,Gas Station,Bank,Fried Chicken Joint,Thai Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


 # part 3: Clustering

### clustering the neigberhood using K means cluster

In [97]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 3, 1, 1, 1, 1, 4])

In [110]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

 merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

In [99]:
scarborough_merged1=scarborough_merged.dropna()
scarborough_merged1

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Bar,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Intersection,Restaurant,Medical Center,Electronics Store,Breakfast Spot,Rental Car Location,Mexican Restaurant,Bank,Clothing Store,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,Grocery Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Playground,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1.0,Hobby Shop,Bus Station,Department Store,Coffee Shop,Chinese Restaurant,Bank,Bar,Grocery Store,General Entertainment,Gas Station
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1.0,Bakery,Bus Line,Ice Cream Shop,Park,Metro Station,Vietnamese Restaurant,Coffee Shop,Gas Station,Gaming Cafe,Fried Chicken Joint
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,3.0,American Restaurant,Motel,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1.0,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant


### Visualization of the clusters

In [101]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        scarborough_merged1['Latitude'], 
        scarborough_merged1['Longitude'], 
        scarborough_merged1['Neighborhood'], 
        scarborough_merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining the clusters
### cluster 1

In [104]:
scarborough_merged1.loc[scarborough_merged1['Cluster Labels'] == 0, scarborough_merged1.columns[[1] + list(range(5, scarborough_merged1.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,0.0,Playground,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
14,Scarborough,0.0,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### cluster 2

In [105]:
scarborough_merged1.loc[scarborough_merged1['Cluster Labels'] == 1, scarborough_merged1.columns[[1] + list(range(5, scarborough_merged1.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1.0,Intersection,Restaurant,Medical Center,Electronics Store,Breakfast Spot,Rental Car Location,Mexican Restaurant,Bank,Clothing Store,Fast Food Restaurant
3,Scarborough,1.0,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Clothing Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant
4,Scarborough,1.0,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,College Stadium,Grocery Store
6,Scarborough,1.0,Hobby Shop,Bus Station,Department Store,Coffee Shop,Chinese Restaurant,Bank,Bar,Grocery Store,General Entertainment,Gas Station
7,Scarborough,1.0,Bakery,Bus Line,Ice Cream Shop,Park,Metro Station,Vietnamese Restaurant,Coffee Shop,Gas Station,Gaming Cafe,Fried Chicken Joint
9,Scarborough,1.0,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,1.0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Gaming Cafe,Light Rail Station,Chinese Restaurant,Bar,Breakfast Spot,Gas Station,Athletics & Sports
11,Scarborough,1.0,Middle Eastern Restaurant,Smoke Shop,Auto Garage,Sandwich Place,Bakery,Breakfast Spot,Vietnamese Restaurant,Electronics Store,College Stadium,Department Store
12,Scarborough,1.0,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,1.0,Pizza Place,Pharmacy,Fast Food Restaurant,Noodle House,Italian Restaurant,Rental Car Location,Gas Station,Bank,Fried Chicken Joint,Thai Restaurant


### Cluster 3

In [106]:
scarborough_merged1.loc[scarborough_merged1['Cluster Labels'] == 2, scarborough_merged1.columns[[1] + list(range(5, scarborough_merged1.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2.0,Bar,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### Cluster 4

In [107]:
scarborough_merged1.loc[scarborough_merged1['Cluster Labels'] == 3, scarborough_merged1.columns[[1] + list(range(5, scarborough_merged1.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,3.0,American Restaurant,Motel,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


### Cluster 5

In [108]:
scarborough_merged1.loc[scarborough_merged1['Cluster Labels'] == 4, scarborough_merged1.columns[[1] + list(range(5, scarborough_merged1.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4.0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store
